<img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">

<h1 style="text-align:center;">CSCI 141, Spring 2019</h1>
<h1 style="text-align:center;"> Polymorphism and inheritance</h1>

In [1]:
# Style the notebook.
from IPython.core.display import HTML
HTML(filename="custom/custom.css")

# Name mangling and inheritance <a id="mangling"/>

We have seen the use of leading double underscores <code>__</code> as a means of hiding the details of a class from prying eyes.

Unfortunately, double underscores do not play nice with inheritance.

Let's start with a simple example:

In [2]:
class Foo (object):
    def __init__(self):
        self.__greeting = ':p :p :p'

class Bar (Foo):
    def print_greeting(self):
        print (self.__greeting)

In [3]:
foobar = Bar()
foobar.print_greeting()

AttributeError: 'Bar' object has no attribute '_Bar__greeting'

## Eh?

Where did the attribute <code>__greeting</code> go?  Why didn't <code>Bar</code> inherit it?



## Name mangling <a id="name_mangling"/>

The answer lies in Python's **name mangling**.

Name mangling refers to altering the user-specified name of something.  This is done for various virtuous purposes, as we will see, and is encountered in other languages such as C++.

To understand what's going on, let's rewrite <code>Foo</code> without any double underscores:

In [4]:
class Foo (object):
    def __init__(self):
        self.greeting = ':p :p :p'
        
class Bar (Foo):
    def print_greeting(self):
        print (self.greeting)

In [5]:
foobar = Bar()
foobar.print_greeting()

:p :p :p


Now things are working as they should.  

Let's build an instance of a <code>Foo</code> and a <code>Bar</code> and look at their attributes.

In [6]:
foo = Foo()
dir(foo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'greeting']

Inside <code>foo</code> we see the attribute named <code>greeting</code>.  Good.

In [7]:
bar = Bar()
dir(bar)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'greeting',
 'print_greeting']

We see that <code>bar</code> has correctly inherited <code>greeting</code>.

Now let's do the same with our original code with the double underscores:

In [8]:
class Foo (object):
    def __init__(self):
        self.__greeting = ':p :p :p'
        
class Bar (Foo):
    def print_greeting(self):
        print (self.__greeting)
        
foo = Foo()
bar = Bar()

In [9]:
dir(foo)

['_Foo__greeting',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

Wait a second &ndash; where is <code>__greeting</code>?!

Instead of <code>__greeting</code> we have <code>_Foo_greeting</code>.  This is Python's name mangling at work.  It adds a prefix generated from the class name to hide <code>__greeting</code> from the outside world.

Now let's look at <code>bar</code>.

In [10]:
dir(bar)

['_Foo__greeting',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'print_greeting']

Now we see where our original problem came from &ndash; when we try to access <code>__greeting</code> in <code>print_greeting</code> in the <code>Bar</code> class, there ss no <code>__greeting</code> to be found.

Instead there is a <code>__Foo_greeting</code>:

In [11]:
print(bar._Foo__greeting)

:p :p :p


## Double underscores in classes trigger name mangling <a id="__"/>

The Python documentation contains [the precise rule on the effect of double underscores](https://docs.python.org/3/reference/lexical_analysis.html#reserved-classes-of-identifiers).

Names beginning with <code>__</code> are
<blockquote>
Class-private names. Names in this category, when used within the context of a class definition, are re-written to use a mangled form to help avoid name clashes between “private” attributes of base and derived classes. See section [Identifiers (Names)](https://docs.python.org/3/reference/expressions.html#atom-identifiers).
</blockquote>

## It's not a bug &ndash; it's a feature!


The effect of double underscores in classes allows you hide attributes in a base class from the derived classes.

This seems annoying at first, since it inhibits your ability to derive new classes from the base class.

But remember that other people can derive classes from your base class, so we're back to the issue of information hiding &ndash; if their derived classes make use of attributes of your base class that change, then changes might break their code.

<div><img style="float: left;" src="images/danger.svg" height="40" width="30">&nbsp;</div> 
Take care when using double underscores in a class if you anticipate deriving other classes from it.

# Constructors and derived classes <a id="constructors"/>

There are a few things you should know about constructors and derived classes.

## A child class without a constructor of its own

Suppose <code>Bar</code> is derived from <code>Foo</code>, but does not have a constructor defined for it:

In [12]:
class Foo (object):
    def __init__(self):
        self.fee = 'fee'
        self.fie = 'fie'
        
class Bar (Foo):
    pass


**Question.**

What do you think the result of the following will be?
```python
foobar = Bar()
print(foobar.fee, foobar.fie)
```

In [13]:
# Let's find out!       
foobar = Bar()
print(foobar.fee, foobar.fie)

fee fie


**Answer.**
<div class="voila">
In this situation Python calls the constructor for the parent class.
</div>

## A child with a constructor of its own.

Now suppose both parent and child class have their own constructors:

In [14]:
class Foo (object):
    def __init__(self):
        self.fee = 'fee'
        self.fie = 'fie'
        
class Bar (Foo):
    def __init__(self):
        self.foe = 'foe'
        self.fum = 'fum'

**Question.**

What do you think the result of the following will be?
```python
        
foobar = Bar()
print(foobar.foe, foobar.fum)
print(foobar.fee, foobar.fie)
```

In [15]:
# Let's find out!        
foobar = Bar()
print(foobar.foe, foobar.fum)
print(foobar.fee, foobar.fie)

foe fum


AttributeError: 'Bar' object has no attribute 'fee'

**Answer.**

<div class="voila">
<p>
Ack! the constructor for the parent class is not being called!
</p>
<p>
We will need to make an explicit call to the parent's constructor.  The <code>super()</code> function allows us access to the parent's methods; in particular, we have overloaded.
</p>
</div>

In [16]:
class Foo (object):
    def __init__(self):
        self.fee = 'fee'
        self.fie = 'fie'
        
class Bar (Foo):
    def __init__(self):
        super().__init__()
        self.foe = 'foe'
        self.fum = 'fum'

The call to ```super().__init__()``` accesses the constructor of <code>Foo</code>.

In [17]:
# Let's find out!        
foobar = Bar()
print(foobar.foe, foobar.fum)
print(foobar.fee, foobar.fie)

foe fum
fee fie


# Another inheritance example

Suppose that we want to create an object to represent a running race. All races will have a few basic attributes: a name and a distance. It would be nice to have accessor methods to return those. We'll assume distances are input in km, so we'll create a method to return the distance in miles as well.

In [ ]:
class Running_Race(object):
    def __init__(self, name, distance_km):
        self.name = name
        self.distance = distance_km
        self.KM_TO_MILES = 0.621
    
    def get_name(self):
        return self.name
    
    def get_distance(self):
        return self.distance
    
    def get_miles(self):
        return self.distance * self.KM_TO_MILES
    

There are specific race types that have pre-determined distances. We could create child classes to represent those. 

In [ ]:
class Marathon(Running_Race):
    def __init__(self,name):
        super().__init__(name, 42)
        self.type = "marathon"
        self.MARATHON_DIST = 26.2
    
    def get_type(self):
        return self.type
    
    def get_miles(self):
       return self.MARATHON_DIST

class Ten_K(Running_Race):
    def __init__(self,name):
        super().__init__(name,10)
        self.type = "10K"
        self.Ten_K_DIST = 6.2
    
    def get_type(self):
        return self.type
    
    def get_miles(self):
        return self.Ten_K_DIST

Let's create a few instances and call the methods we have. Which, if any, of the method calls below will create an error? Why?

In [ ]:
Shamrock = Marathon("Shamrock")
print(Shamrock.get_type())
print(Shamrock.get_name())
print(Shamrock.get_distance())
print(Shamrock.get_miles())

Peachtree = Ten_K("Peachtree Road Race")
print(Peachtree.get_type())
print(Peachtree.get_miles())

Shoreline  = Running_Race("Shoreline Classic", 15)
print(Shoreline.get_type())
print(Shoreline.get_distance())
print(Shoreline.get_miles())


Some races can also be done as relays, where the total distance is completed by more than one runner. Let's create a child class of <code>Marathon</code> that is a relay.

In [ ]:
class Marathon_Relay(Marathon):
    def __init__(self, name, number_runners):
        super().__init__(name)
        self.__runners = number_runners #Why did we make this one private?
        
    def get_runners(self):
        return self.__runners 
    

In [ ]:
my_race = Marathon_Relay("Relay Magic!", 4)
print(my_race.get_runners())
print(my_race.get_miles())
print(my_race.get_name())
print(my_race.get_distance())
print(my_race.get_type())


We previously created a <code>Marathon</code> object called Shamrock. Why does the method call below produce an error?

In [ ]:
print(Shamrock.get_runners())

It can be helpful to draw a diagram to display inheritance. Parent classes are located above their child classes and are connected with arrows.


<div><img style="float: left;" src="images/running.png" height="400" width="800">&nbsp;</div>